In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

In [2]:
# MODIFY THIS LINE!!!
# Step 1: Save raw COD files from all campuses into separate directory to be referenced below
# Set path of directory with saved COD files and define all data file names from all campuses
# Set string aid_year long description based on the first DL filename
path =r'C:/Users/mjc341/Desktop/DirectLoans/08.13 Update'
path_final =r'C:/Users/mjc341/Desktop/DirectLoans/FinalVersions'
filenames = glob.glob(path + "/dsyh18op.*")

In [3]:
# Read data from individual COD DL csv files from all campuses
dl_csv_files = []
for filename in filenames:
    dl_csv_files.append(pd.read_csv(filename, skiprows=5, skipfooter=1, engine='python', index_col=False))

In [4]:
aid_year = '20' + str((int(filename[-8:-6])-1)) + '-20' + filename[-8:-6]
date = path[-12:-7]

In [5]:
# Aid Year string for DL Reconciliation
aid_year

'2017-2018'

In [6]:
date

'08.13'

In [7]:
# Combine data from all campuses into one DataFrame
dl_combined = pd.concat(dl_csv_files, ignore_index=True)

In [8]:
# Select records from COD DL combined file with Total Net Amt > 0
dl_gt_0 = dl_combined[dl_combined['TOTAL NET AMT']>0]

In [9]:
# Write COD DL file to Excel with all campuses combined
dl_gt_0.to_excel(path + "/dl_combined_" + filename[-8:-6] + ".xlsx", sheet_name='COD All Campuses', index=False)

In [10]:
# MODIFY THIS LINE TEMPORARILY UNTIL LOGIC IS CODED
# Open CaneLink DL Disbursement file regardless of the suffix added to filename by CaneLink Query Manager
canelink_dl_disb = pd.read_excel(path + "/AG_ALL_DISB_BY_YEAR_ALL_LOAN_8_13_18.xlsx", skiprows=1, index_col=False)

In [11]:
len(canelink_dl_disb)

15402

In [12]:
# Rename key field in COD DL file to match label in CaneLink DL disbursements file
dl_booked = dl_gt_0.rename(columns={'AWARD IDENTIFIER': 'Ln App ID'})

In [13]:
len(dl_booked)

12579

In [14]:
canelink_dl_disb.head(3)

,Ln App ID,ID,Aid Yr,SUM(A.DISBURSED_BALANCE)
0,187664130U18G01536001,54259298,2018,6760
1,187664130P18G01536001,54259298,2018,12405
2,245694060P18G01536001,54262481,2018,12405


In [15]:
# Bring DOE total net amt into CL file and join by Ln App ID
dl_compare = canelink_dl_disb.merge(dl_booked[['Ln App ID','TOTAL NET AMT']],on="Ln App ID",how='left')

In [16]:
#Replace NaN values with 0 for total net amt
dl_compare = dl_compare.fillna(value={'TOTAL NET AMT':0})

In [17]:
len(dl_compare)

15402

In [18]:
dl_compare.head(3)

,Ln App ID,ID,Aid Yr,SUM(A.DISBURSED_BALANCE),TOTAL NET AMT
0,187664130U18G01536001,54259298,2018,6760,6760.0
1,187664130P18G01536001,54259298,2018,12405,12405.0
2,245694060P18G01536001,54262481,2018,12405,12405.0


In [19]:
# Add discrepancy column (difference between CL disbursed balance and DL Total net amt)
dl_compare['DISCREPANCY'] = dl_compare['SUM(A.DISBURSED_BALANCE)'] - dl_compare['TOTAL NET AMT']

In [20]:
dl_compare.head(3)

,Ln App ID,ID,Aid Yr,SUM(A.DISBURSED_BALANCE),TOTAL NET AMT,DISCREPANCY
0,187664130U18G01536001,54259298,2018,6760,6760.0,0.0
1,187664130P18G01536001,54259298,2018,12405,12405.0,0.0
2,245694060P18G01536001,54262481,2018,12405,12405.0,0.0


In [21]:
len(dl_compare)

15402

In [22]:
# Coding for Match Status column
dl_compare.loc[dl_compare['DISCREPANCY'] == 0, 'Match Status'] = 'Booked, Matched'
dl_compare.loc[(dl_compare['DISCREPANCY'] > 0) & (dl_compare['TOTAL NET AMT'] == 0), 'Match Status'] = 'Unbooked'
dl_compare.loc[(dl_compare['DISCREPANCY'] > 0) & (dl_compare['TOTAL NET AMT'] != 0) & (dl_compare['SUM(A.DISBURSED_BALANCE)'] != 0), 'Match Status'] = 'Booked, Mismatched'
dl_compare.loc[dl_compare['DISCREPANCY'] < 0, 'Match Status'] = 'Booked, Other'

In [23]:
dl_compare.head(49)

,Ln App ID,ID,Aid Yr,SUM(A.DISBURSED_BALANCE),TOTAL NET AMT,DISCREPANCY,Match Status
0,187664130U18G01536001,54259298,2018,6760,6760.0,0.0,"Booked, Matched"
1,187664130P18G01536001,54259298,2018,12405,12405.0,0.0,"Booked, Matched"
2,245694060P18G01536001,54262481,2018,12405,12405.0,0.0,"Booked, Matched"
3,766823723P18E00532001,54101079,2018,0,0.0,0.0,"Booked, Matched"
4,154966597S18G01536001,54258445,2018,4211,4211.0,0.0,"Booked, Matched"
5,590496241U18G01536001,53229574,2018,4702,4702.0,0.0,"Booked, Matched"
6,593418860U18G01536001,54205352,2018,1980,1980.0,0.0,"Booked, Matched"
7,061472352S18G01536001,54206710,2018,5442,5442.0,0.0,"Booked, Matched"
8,770726772S18G01536001,54208002,2018,1238,1238.0,0.0,"Booked, Matched"
9,591296324U18G01536001,54238786,2018,6928,6928.0,0.0,"Booked, Matched"


In [24]:
# Coding for deriving Campus from Ln App ID
dl_compare.loc[dl_compare['Ln App ID'].str[12:18] == 'G01536', 'Campus'] = 'Gables'
dl_compare.loc[dl_compare['Ln App ID'].str[12:18] == 'E00532', 'Campus'] = 'Law'
dl_compare.loc[dl_compare['Ln App ID'].str[12:18] == 'E00533', 'Campus'] = 'Med'

In [25]:
# Coding for deriving School Code from Ln App ID
dl_compare['School Code'] = dl_compare['Ln App ID'].str[12:18]

In [26]:
# Coding for deriving Career from Ln App ID
dl_compare.loc[dl_compare['Ln App ID'].str[12:18] == 'G01536', 'Career'] = 'UGRD/GRAD'
dl_compare.loc[dl_compare['Ln App ID'].str[12:18] == 'E00532', 'Career'] = 'LAW'
dl_compare.loc[dl_compare['Ln App ID'].str[12:18] == 'E00533', 'Career'] = 'MED'

In [27]:
dl_compare.head(8)

,Ln App ID,ID,Aid Yr,SUM(A.DISBURSED_BALANCE),TOTAL NET AMT,DISCREPANCY,Match Status,Campus,School Code,Career
0,187664130U18G01536001,54259298,2018,6760,6760.0,0.0,"Booked, Matched",Gables,G01536,UGRD/GRAD
1,187664130P18G01536001,54259298,2018,12405,12405.0,0.0,"Booked, Matched",Gables,G01536,UGRD/GRAD
2,245694060P18G01536001,54262481,2018,12405,12405.0,0.0,"Booked, Matched",Gables,G01536,UGRD/GRAD
3,766823723P18E00532001,54101079,2018,0,0.0,0.0,"Booked, Matched",Law,E00532,LAW
4,154966597S18G01536001,54258445,2018,4211,4211.0,0.0,"Booked, Matched",Gables,G01536,UGRD/GRAD
5,590496241U18G01536001,53229574,2018,4702,4702.0,0.0,"Booked, Matched",Gables,G01536,UGRD/GRAD
6,593418860U18G01536001,54205352,2018,1980,1980.0,0.0,"Booked, Matched",Gables,G01536,UGRD/GRAD
7,061472352S18G01536001,54206710,2018,5442,5442.0,0.0,"Booked, Matched",Gables,G01536,UGRD/GRAD


In [28]:
len(dl_compare)

15402

In [29]:
# Select Unbooked and Mismatched loans only
dl_unbooked_mismatched = dl_compare.loc[(dl_compare['Match Status'] == "Unbooked") | 
                                        (dl_compare['Match Status'] == "Booked, Mismatched")]

In [30]:
len(dl_unbooked_mismatched)

971

In [31]:
dl_unbooked_mismatched.head(8)

,Ln App ID,ID,Aid Yr,SUM(A.DISBURSED_BALANCE),TOTAL NET AMT,DISCREPANCY,Match Status,Campus,School Code,Career
51,591175920U18G01536001,54216799,2018,6760,0.0,6760.0,Unbooked,Gables,G01536,UGRD/GRAD
226,768981199S18G01536001,54257415,2018,4452,2226.0,2226.0,"Booked, Mismatched",Gables,G01536,UGRD/GRAD
233,595398158U18E00533001,54180548,2018,40068,0.0,40068.0,Unbooked,Med,E00533,MED
234,291922680U18E00533001,54182199,2018,40068,0.0,40068.0,Unbooked,Med,E00533,MED
235,589411465U18E00533001,54183098,2018,40068,0.0,40068.0,Unbooked,Med,E00533,MED
236,589411465P18E00533001,54183098,2018,14698,0.0,14698.0,Unbooked,Med,E00533,MED
238,589517034U18E00533001,54229972,2018,44464,0.0,44464.0,Unbooked,Med,E00533,MED
239,591276910U18E00533001,54231895,2018,42266,0.0,42266.0,Unbooked,Med,E00533,MED


In [32]:
# Select loans booked in COD DL file that have any amount not disbursed in CaneLink
dl_cod_not_in_canelink = dl_compare.loc[dl_compare['Match Status'] == "Booked, Other"]

In [33]:
len(dl_cod_not_in_canelink)

13

In [34]:
dl_cod_not_in_canelink.head(3)

,Ln App ID,ID,Aid Yr,SUM(A.DISBURSED_BALANCE),TOTAL NET AMT,DISCREPANCY,Match Status,Campus,School Code,Career
1169,594148617P18G01536003,54265781,2018,12472,13045.0,-573.0,"Booked, Other",Gables,G01536,UGRD/GRAD
2548,139022216U18G01536001,50430244,2018,1420,1984.0,-564.0,"Booked, Other",Gables,G01536,UGRD/GRAD
3626,766348721P18G01536002,54188585,2018,19040,19655.0,-615.0,"Booked, Other",Gables,G01536,UGRD/GRAD


In [35]:
dl_cod_not_in_canelink.pivot_table(index=["Match Status"],values=["DISCREPANCY"],
                       columns=["Campus"], aggfunc=[np.sum,len],margins=True)

sum                  len      
              DISCREPANCY          DISCREPANCY      
Campus             Gables      All      Gables   All
Match Status                                        
Booked, Other    -42474.0 -42474.0        13.0  13.0
All              -42474.0 -42474.0        13.0  13.0

In [36]:
len(dl_compare["TOTAL NET AMT"])  

15402

In [37]:
summary_prep = pd.DataFrame(data=dl_compare.groupby(["Campus"])["SUM(A.DISBURSED_BALANCE)","TOTAL NET AMT",
                                                                   "DISCREPANCY"].sum())

In [38]:
summary_prep

,SUM(A.DISBURSED_BALANCE),TOTAL NET AMT,DISCREPANCY
Campus,,,
Gables,110031302,109651679.0,379623.0
Law,35772530,35539055.0,233475.0
Med,24875591,0.0,24875591.0


In [39]:
summary_prep = pd.DataFrame(data=dl_compare.groupby(["Campus"])["SUM(A.DISBURSED_BALANCE)","TOTAL NET AMT",
                                                                   "DISCREPANCY"].sum()).T

In [40]:
summary_prep["TOTAL"] = summary_prep["Gables"]+summary_prep["Law"]+summary_prep["Med"]

In [41]:
summary_prep.index.names = ['AY '+aid_year+' as of '+date[1:5]]

In [42]:
summary_prep.rename(index={'SUM(A.DISBURSED_BALANCE)': 'CaneLink DL Disbursed Amt',
                           'TOTAL NET AMT': 'COD Total Net Amt for DL Disbursed per DOE',
                    'DISCREPANCY': 'Net Discrepancy between COD and CaneLink'}, inplace=True)

In [43]:
summary_prep

Campus,Gables,Law,Med,TOTAL
AY 2017-2018 as of 8.13,,,,
CaneLink DL Disbursed Amt,110031302.0,35772530.0,24875591.0,170679423.0
COD Total Net Amt for DL Disbursed per DOE,109651679.0,35539055.0,0.0,145190734.0
Net Discrepancy between COD and CaneLink,379623.0,233475.0,24875591.0,25488689.0


In [44]:
summary_prep2_count = pd.DataFrame(data=dl_unbooked_mismatched.groupby(["Campus"])["DISCREPANCY"].count()).T
summary_prep2_sum = pd.DataFrame(data=dl_unbooked_mismatched.groupby(["Campus"])["DISCREPANCY"].sum()).T

In [45]:
summary_prep2_count

Campus,Gables,Law,Med
DISCREPANCY,56,20,895


In [46]:
summary_prep2_sum

Campus,Gables,Law,Med
DISCREPANCY,422097.0,233475.0,24875591.0


In [47]:
# To control for campuses where no data exists for unbooked/mismatched loans
summary_prep2_count["Gables"] = dl_unbooked_mismatched.loc[dl_unbooked_mismatched['Campus'] == "Gables"].count()
summary_prep2_sum["Gables"] = dl_unbooked_mismatched.loc[dl_unbooked_mismatched['Campus'] == "Gables"].sum()
summary_prep2_count["Med"] = dl_unbooked_mismatched.loc[dl_unbooked_mismatched['Campus'] == "Med"].count()
summary_prep2_sum["Med"] = dl_unbooked_mismatched.loc[dl_unbooked_mismatched['Campus'] == "Med"].sum()
summary_prep2_count["Law"] = dl_unbooked_mismatched.loc[dl_unbooked_mismatched['Campus'] == "Law"].count()
summary_prep2_sum["Law"] = dl_unbooked_mismatched.loc[dl_unbooked_mismatched['Campus'] == "Law"].sum()

In [48]:
summary_prep2_count

Campus,Gables,Law,Med
DISCREPANCY,56,20,895


In [49]:
summary_prep2_sum

Campus,Gables,Law,Med
DISCREPANCY,422097,233475,2.48756e+07


In [50]:
summary_prep2_count["TOTAL"] = summary_prep2_count["Gables"]+summary_prep2_count["Law"]+summary_prep2_count["Med"]
summary_prep2_sum["TOTAL"] = summary_prep2_sum["Gables"]+summary_prep2_sum["Law"]+summary_prep2_sum["Med"]

In [51]:
summary_prep2_count.index.names = ['AY '+aid_year+' as of '+date[1:5]]
summary_prep2_sum.index.names = ['AY '+aid_year+' as of '+date[1:5]]

In [52]:
summary_prep2_count.rename(index={'DISCREPANCY': 'Unbooked + Mismatched Loan Count'}, inplace=True)
summary_prep2_sum.rename(index={'DISCREPANCY': 'Sum of Unbooked + Mismatched Loan Disbursements'}, inplace=True)

In [53]:
summary_prep2_count

Campus,Gables,Law,Med,TOTAL
AY 2017-2018 as of 8.13,,,,
Unbooked + Mismatched Loan Count,56,20,895,971


In [54]:
summary_prep2_sum

Campus,Gables,Law,Med,TOTAL
AY 2017-2018 as of 8.13,,,,
Sum of Unbooked + Mismatched Loan Disbursements,422097,233475,2.48756e+07,2.55312e+07


In [55]:
summary_prep3_count = pd.DataFrame(data=dl_cod_not_in_canelink.groupby(["Campus"])["DISCREPANCY"].count()).T
summary_prep3_sum = pd.DataFrame(data=dl_cod_not_in_canelink.groupby(["Campus"])["DISCREPANCY"].sum()).T

In [56]:
# To control for campuses where no data exists for COD disbursed and not in CaneLink
summary_prep3_count["Gables"] = dl_cod_not_in_canelink.loc[dl_cod_not_in_canelink['Campus'] == "Gables"].count()
summary_prep3_sum["Gables"] = dl_cod_not_in_canelink.loc[dl_cod_not_in_canelink['Campus'] == "Gables"].sum()
summary_prep3_count["Med"] = dl_cod_not_in_canelink.loc[dl_cod_not_in_canelink['Campus'] == "Med"].count()
summary_prep3_sum["Med"] = dl_cod_not_in_canelink.loc[dl_cod_not_in_canelink['Campus'] == "Med"].sum()
summary_prep3_count["Law"] = dl_cod_not_in_canelink.loc[dl_cod_not_in_canelink['Campus'] == "Law"].count()
summary_prep3_sum["Law"] = dl_cod_not_in_canelink.loc[dl_cod_not_in_canelink['Campus'] == "Law"].sum()

In [57]:
summary_prep3_count["TOTAL"] = summary_prep3_count["Gables"]+summary_prep3_count["Law"]+summary_prep3_count["Med"]
summary_prep3_sum["TOTAL"] = summary_prep3_sum["Gables"]+summary_prep3_sum["Law"]+summary_prep3_sum["Med"]

In [58]:
summary_prep3_count.index.names = ['AY '+aid_year+' as of '+date[1:5]]
summary_prep3_sum.index.names = ['AY '+aid_year+' as of '+date[1:5]]

In [59]:
summary_prep3_count.rename(index={'DISCREPANCY': 'Number of loans with COD disbursements not in PS'}, inplace=True)
summary_prep3_sum.rename(index={'DISCREPANCY': 'Sum of COD disbursements not in PS'}, inplace=True)

In [60]:
summary_prep3_count

Campus,Gables,Med,Law,TOTAL
AY 2017-2018 as of 8.13,,,,
Number of loans with COD disbursements not in PS,13,0,0,13


In [61]:
summary_prep3_sum

Campus,Gables,Med,Law,TOTAL
AY 2017-2018 as of 8.13,,,,
Sum of COD disbursements not in PS,-42474,0.0,0.0,-42474


In [62]:
summary_concat = [summary_prep, summary_prep2_count, summary_prep2_sum, summary_prep3_count, summary_prep3_sum]
dl_summary = pd.concat(summary_concat)

In [63]:
dl_summary

,Gables,Law,Med,TOTAL
AY 2017-2018 as of 8.13,,,,
CaneLink DL Disbursed Amt,1.10031e+08,3.57725e+07,2.48756e+07,1.70679e+08
COD Total Net Amt for DL Disbursed per DOE,1.09652e+08,3.55391e+07,0,1.45191e+08
Net Discrepancy between COD and CaneLink,379623,233475,2.48756e+07,2.54887e+07
Unbooked + Mismatched Loan Count,56,20,895,971
Sum of Unbooked + Mismatched Loan Disbursements,422097,233475,2.48756e+07,2.55312e+07
Number of loans with COD disbursements not in PS,13,0,0,13
Sum of COD disbursements not in PS,-42474,0,0,-42474


In [64]:
writer = pd.ExcelWriter(path_final + '/' + aid_year + ' Direct Loan Disbursement Reconciliation - 2018.' + date + '.xlsx')
dl_summary.to_excel(writer,'Summary')
dl_unbooked_mismatched.to_excel(writer,'Unbooked+Mismatched by Ln_ID', index=False)
dl_cod_not_in_canelink.to_excel(writer,'CoD > CaneLink by Ln_ID', index=False)
dl_compare.to_excel(writer,'All Loans by Ln ID', index=False)
canelink_dl_disb.to_excel(writer,'Orig CS Query', index=False)

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet = writer.sheets['Summary']
worksheet2 = writer.sheets['Unbooked+Mismatched by Ln_ID']
worksheet3 = writer.sheets['CoD > CaneLink by Ln_ID']
worksheet4 = writer.sheets['All Loans by Ln ID']
worksheet5 = writer.sheets['Orig CS Query']

# Add some cell formats.
format1 = workbook.add_format({'num_format': '$#,##0.00', 'bold': True, 'border': '1'})
format2 = workbook.add_format({'num_format': '$#,##0.00', 'border': '1'})
format3 = workbook.add_format({'bold': True, 'font_color': 'orange', 'border': '1'})
format4 = workbook.add_format({'num_format': '$#,##0.00', 'bold': True, 'font_color': 'orange', 'border': '1'})
format5 = workbook.add_format({'border': '1'})

# Note: It isn't possible to format any cells that already have a format such
# as the index or headers or any cells that contain dates or datetimes.

# Set the column width and format.
worksheet.set_column('A:A', 51)
worksheet.set_column('B:E', 15)
worksheet.set_row(1, None, format2)
worksheet.set_row(2, None, format2)
worksheet.set_row(3, None, format1)
worksheet.set_row(4, None, format3)
worksheet.set_row(5, None, format4)
worksheet.set_row(6, None, format5)
worksheet.set_row(7, None, format2)
worksheet2.set_column('A:A', 22)
worksheet2.set_column('B:C', 18)
worksheet2.set_column('D:D', 27)
worksheet2.set_column('E:J', 18)
worksheet3.set_column('A:A', 22)
worksheet3.set_column('B:C', 18)
worksheet3.set_column('D:D', 27)
worksheet3.set_column('E:J', 18)
worksheet4.set_column('A:A', 22)
worksheet4.set_column('B:C', 18)
worksheet4.set_column('D:D', 27)
worksheet4.set_column('E:J', 18)
worksheet5.set_column('A:A', 22)
worksheet5.set_column('B:C', 18)
worksheet5.set_column('D:D', 27)
worksheet5.set_column('E:J', 18)
#worksheet.write(4, 1, None, format3)
#worksheet.write(5, 1, None, format4)

# Set the format but not the column width.
# worksheet.set_column('C:C', None, format2)

writer.save()